# Inputting only s11 and freq and outputting all geometry parameters

### Regular TensorFlow 

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from matplotlib import rc

/tmp/ipykernel_455850/3602695761.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-03-24 16:39:27.288249: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-24 16:39:27.317948: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 16:3

In [2]:
# Import data and preprocess
df = pd.read_csv("../test_data/new leaky wave/S11_V1.csv")
# df = pd.read_csv("../test_data/Grounded CPW Leaky Wave antenna/S11 Data.csv")
df = df.drop(df[df['dB(S(1,1)) []'] > 0].index) # Remove all rows with positive s11

# # Split into x and y
# input_x = df.drop(columns=['dB(S(1,1)) []'], axis=1)
# input_y = df[['dB(S(1,1)) []']]

# Split data into training and testing
train_df = df.sample(frac=0.8, random_state=0)
test_df = df.drop(train_df.index)
training_cols = ['dB(S(1,1)) []', 'Freq [GHz]']
X_train = train_df[training_cols]
X_test = test_df[training_cols]
y_train = train_df.drop(columns=training_cols)
y_test = test_df.drop(columns=training_cols)

In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() # Initialize scaler
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [4]:
X_train

,"dB(S(1,1)) []",Freq [GHz]
221,-7.026838,12.71
3460,-9.517524,13.34
154,-9.496576,15.77
5053,-2.055686,11.27
1220,-1.628387,11.72
...,...,...
4829,-16.261329,18.38
2950,-8.574691,12.89
512,-1.596860,11.63
810,-1.347443,11.18


In [5]:
y_train

,cpw_in [mm],feed_l [mm],ground_w [mm],patch_ground_w [mm],patch_l [mm]
221,2.5,3.0,0.75,0.8,3.0
3460,1.5,3.5,1.00,1.2,3.0
154,2.0,3.0,0.75,0.8,3.0
5053,1.5,3.0,1.25,1.0,3.5
1220,1.5,3.0,1.00,1.0,3.0
...,...,...,...,...,...
4829,1.5,3.0,1.00,1.0,3.5
2950,1.5,3.5,1.25,0.8,3.0
512,2.5,3.0,1.00,0.8,3.0
810,2.5,3.0,1.25,0.8,3.0


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

inputs = Input(shape=(2,))
hidden1 = Dense(160, activation='relu')(inputs)
hidden2 = Dense(160, activation='relu')(hidden1)
hidden3 = Dense(160, activation='relu')(hidden2)
hidden4 = Dense(160, activation='relu')(hidden3)

outputs = Dense(5, activation='linear')(hidden4)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense (Dense)               (None, 160)               480       
                                                                 
 dense_1 (Dense)             (None, 160)               25760     
                                                                 
 dense_2 (Dense)             (None, 160)               25760     
                                                                 
 dense_3 (Dense)             (None, 160)               25760     
                                                                 
 dense_4 (Dense)             (None, 5)                 805       
                                                                 
Total params: 78565 (306.89 KB)
Trainable params: 78565 (306.

2024-03-24 16:39:34.309252: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1030 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:02:00.0, compute capability: 8.6


In [7]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


history = model.fit(X_train_scaled, y_train, epochs=150, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/150


2024-03-24 16:39:39.817078: I external/local_xla/xla/service/service.cc:168] XLA service 0x990cc00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-24 16:39:39.817113: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-03-24 16:39:39.822847: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-24 16:39:39.840642: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1711312779.909446  455956 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


182/182 [==============================] - 2s 3ms/step - loss: 0.5216 - accuracy: 0.5841 - val_loss: 0.0734 - val_accuracy: 0.5863
Epoch 2/150
182/182 [==============================] - 0s 2ms/step - loss: 0.0743 - accuracy: 0.5993 - val_loss: 0.0741 - val_accuracy: 0.5863
Epoch 3/150
182/182 [==============================] - 0s 2ms/step - loss: 0.0723 - accuracy: 0.5993 - val_loss: 0.0731 - val_accuracy: 0.5863
Epoch 4/150
182/182 [==============================] - 0s 2ms/step - loss: 0.0713 - accuracy: 0.5993 - val_loss: 0.0737 - val_accuracy: 0.5863
Epoch 5/150
182/182 [==============================] - 0s 2ms/step - loss: 0.0720 - accuracy: 0.5993 - val_loss: 0.0723 - val_accuracy: 0.5863
Epoch 6/150
182/182 [==============================] - 0s 2ms/step - loss: 0.0716 - accuracy: 0.5993 - val_loss: 0.0715 - val_accuracy: 0.5863
Epoch 7/150
182/182 [==============================] - 0s 2ms/step - loss: 0.0710 - accuracy: 0.5993 - val_loss: 0.0725 - val_accuracy: 0.5863
Epoch 8/150

In [9]:
predictions = model.predict(X_test_scaled)

57/57 [==============================] - 0s 795us/step


In [10]:
pd.DataFrame(predictions)

,0,1,2,3,4
0,1.945936,3.075330,0.981861,1.010613,3.393495
1,1.918496,2.932764,0.976014,0.906855,3.467584
2,1.909541,2.938926,0.978484,0.907691,3.479191
3,1.774505,3.024738,0.990672,0.941621,3.422717
4,1.825775,2.983185,0.983253,0.930079,3.465032
...,...,...,...,...,...
1813,2.560113,3.016049,1.066854,0.959821,3.577269
1814,2.564344,3.021035,1.069109,0.964178,3.579599
1815,2.491637,3.006498,1.045354,0.967699,3.523133
1816,2.400003,3.076308,1.040778,0.981020,3.567334


In [11]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, predictions)

0.06751964318075765

### Keras Tuner

In [22]:
import keras_tuner
class AntennaModel(keras_tuner.HyperModel):
    def __init__(self, input_shape):
        self.input_shape= input_shape
        
    def build(self, hp):
        model= keras.Sequential()

        for i in range(hp.Int('num_layers', 1, 4)):
            model.add(keras.layers.Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32), activation='relu', input_shape=self.input_shape))
        model.add(keras.layers.Dense(5)) # Output layer

        # Tune the learning rate for the optimizer 
        hp_learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=0.1)

        model.compile(loss='mse',
                    optimizer= keras.optimizers.Adam(learning_rate=hp_learning_rate),
                    metrics= ['mse']
                     )

        return model

In [23]:
import keras_tuner
antenna_model = AntennaModel(input_shape=[len(X_train.keys())])
tuner = keras_tuner.RandomSearch(
    antenna_model,
    objective='val_mse',
    max_trials=3,
    executions_per_trial=1)

tuner.search_space_summary()
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(X_train_scaled, y_train, epochs=100, validation_split=0.2, validation_data = (X_test_scaled, y_test), callbacks=[stop_early])

Trial 3 Complete [00h 00m 04s]
val_mse: 0.07274043560028076

Best val_mse So Far: 0.0682472512125969
Total elapsed time: 00h 00m 20s


In [24]:
best_model = tuner.get_best_models(num_models=1)[0]
tuner_preds = best_model.predict(X_test_scaled)
mean_squared_error(y_test, tuner_preds)


57/57 [==============================] - 0s 856us/step


0.06824724479820463

### Sklearn

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor

model = MultiOutputRegressor(
    RandomForestRegressor(max_depth=85, min_samples_leaf=1, min_samples_split=2, n_estimators=1000
))
scaler = StandardScaler() # Initialize scaler
pipeline = Pipeline(steps=[('normalize', scaler), ('model', model)]) # Create pipeline with scaler and model


In [26]:
pipeline.fit(X_train_scaled,y_train)

Pipeline(steps=[('normalize', StandardScaler()),
                ('model',
                 MultiOutputRegressor(estimator=RandomForestRegressor(max_depth=85,
                                                                      n_estimators=1000)))])

In [27]:
sklearn_pred = pipeline.predict(X_test_scaled)

In [28]:
mean_squared_error(y_test, sklearn_pred)

0.07705865244169437